# **Recruit Restaurant Data Analysis**
___
This notebook aims at providing a first glance at the data provided for the "Recruit Restaurant Visitor Forecast" challenge. The purpose of this competition is to predict the number of people that will make reservations during 5 weeks, from April 2017 to May 2017. Historical data for a period $\sim$1 year prior to April 2017 is provided in order get some insights on customers' habits.

## 1. Load the data

First step will be to load all the necessary packages and libraries

In [3]:
import numpy as np
import datetime
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math, warnings
from mpl_toolkits.basemap import Basemap
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings('ignore')

Files provided for the competition are listed below:

In [4]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
file_list = check_output(["ls", "../input"]).decode("utf8")
file_list = file_list.strip().split('\n')

As a first step, it would be good to examine the variable types and the number of null values. Variables corresponding to date-time data will be converted to `datetime` format for the purpose of our analysis.

In [5]:
def get_info(df):
    print('Shape:',df.shape)
    print('Size: {:5.2f} MB'.format(df.memory_usage().sum()/1024**2))
    tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
    tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values'}))
    tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
    display(tab_info)
    display(df[:5])

In [6]:
#Display info on the csv files along with their file sizes
for index, file in enumerate(file_list):
    var_name = file.rstrip('.csv')
    print(file)
    locals()[var_name] = pd.read_csv('../input/'+file)
    
    #convert to datetime
    for col in locals()[var_name].columns:
        if col.endswith('datetime') or col.endswith('date'):
            locals()[var_name][col] = pd.to_datetime(locals()[var_name][col])
    #get info
    get_info(locals()[var_name])

A brief description of the files based on information provided in the competition's home page:
- **air_reserve.csv, hpg_reserve.csv**: reservations made in the air (*AirREGI*) or hpg (*Hot Pepper Gourmet*) systems
- **air_store_info.csv, hpg_store_info.csv**: information about *air* and *hpg* restaurants
- **store_id_relation.csv**: link the restaurants ids of the air and hpg systems
- **air_visit_data.csv**: historical visit data for the air restaurants
- **date_info.csv**: information about the calendar dates in the dataset

___
## 2. Restaurant locations

Putting the restaurant locations on a map. The plot contains data from both the hpg and air store reserves tables. However, it is important to keep in mind that the locations are only approximations in order to keep the data anonymous.

In [7]:
def draw_map(df, title):
    plt.figure(figsize=(11,6))
    map = Basemap(resolution='i',llcrnrlon=127, urcrnrlon=147,
                  llcrnrlat=29, urcrnrlat=47, lat_0=0, lon_0=0,)
    map.shadedrelief()
    map.drawcoastlines()
    map.drawcountries(linewidth = 3)
    map.drawstates(color='0.3')
    parallels = np.arange(0.,360,10.,)
    map.drawparallels(parallels, labels = [True for s in range(len(parallels))])
    meridians = np.arange(0.,360,10.,)
    map.drawmeridians(meridians, labels = [True for s in range(len(meridians))])
    
    #visualizing the locations on a map
    for index, (y,x) in df[['latitude','longitude']].iterrows():
        x, y = map(x, y)
        map.plot(x, y, marker='o', markersize = 5, markeredgewidth = 1, color = 'red',
                 markeredgecolor='k')
    plt.title(title, y = 1.05)

In [9]:
#plotting location data from the air_store_info table
airstore = pd.read_csv("../input/air_store_info.csv.zip")
draw_map(airstore, 'air store restaurant locations')

___
## 3. Reservations against visists

Defining a utility class that I will subsequently use to generate all the plots.


In [ ]:
class Figure_style():
    #initializing
    def __init__(self, size_x = 11, size_y = 5, nrows = 1, ncols = 1):
        sns.set_style("white")
        sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})
        self.fig, axs = plt.subplots(nrows = nrows, ncols = ncols, figsize=(size_x,size_y,))
        
        #converting self.axs to 2D array
        if nrows == 1 and ncols == 1:
            self.axs = np.reshape(axs, (1, -1))
        elif nrows == 1:
            self.axs = np.reshape(axs, (1, -1))
        elif ncols == 1:
            self.axs = np.reshape(axs, (-1, 1))
   
    def pos_update(self, ix, iy):
        self.ix, self.iy = ix, iy
    
    def style(self):
        self.axs[self.ix, self.iy].spines['right'].set_visible(False)
        self.axs[self.ix, self.iy].spines['top'].set_visible(False)
        self.axs[self.ix, self.iy].yaxis.grid(color='lightgray', linestyle=':')
        self.axs[self.ix, self.iy].xaxis.grid(color='lightgray', linestyle=':')
        self.axs[self.ix, self.iy].tick_params(axis='both', which='major',
                                               labelsize=10, size = 5)
    
    #apply legends to the plots
    def draw_legend(self, location='upper right'):
        legend = self.axs[self.ix, self.iy].legend(loc = location, shadow=True,
                                        facecolor = 'g', frameon = True)
        legend.get_frame().set_facecolor('whitesmoke')
    
    def cust_plot(self, x, y, color='b', linestyle='-', linewidth=1, marker=None, label=''):
        if marker:
            markerfacecolor, marker, markersize = marker[:]
            self.axs[self.ix, self.iy].plot(x, y, color = color, linestyle = linestyle,
                                linewidth = linewidth, marker = marker, label = label,
                                markerfacecolor = markerfacecolor, markersize = markersize)
        else:
            self.axs[self.ix, self.iy].plot(x, y, color = color, linestyle = linestyle,
                                        linewidth = linewidth, label=label)
        self.fig.autofmt_xdate()
    
    #plotting dates
    def cust_plot_date(self, x, y, color='lightblue', linestyle='-',
                       linewidth=1, markeredge=False, label=''):
        markeredgewidth = 1 if markeredge else 0
        self.axs[self.ix, self.iy].plot_date(x, y, color='lightblue', markeredgecolor='grey',
                                  markeredgewidth = markeredgewidth, label=label)
    
    #scatterplots
    def cust_scatter(self, x, y, color = 'lightblue', markeredge = False, label=''):
        markeredgewidth = 1 if markeredge else 0
        self.axs[self.ix, self.iy].scatter(x, y, color=color,  edgecolor='grey',
                                  linewidths = markeredgewidth, label=label)    
    #setting xlabels
    def set_xlabel(self, label, fontsize = 14):
        self.axs[self.ix, self.iy].set_xlabel(label, fontsize = fontsize)
        
    #setting ylabels
    def set_ylabel(self, label, fontsize = 14):
        self.axs[self.ix, self.iy].set_ylabel(label, fontsize = fontsize)
        
    #setting x-limits
    def set_xlim(self, lim_inf, lim_sup):
        self.axs[self.ix, self.iy].set_xlim([lim_inf, lim_sup])
        
    #setting y-limits
    def set_ylim(self, lim_inf, lim_sup):
        self.axs[self.ix, self.iy].set_ylim([lim_inf, lim_sup])  

Selecting restaurant id's which are common to both the hpg reserve and air store databases.

In [ ]:
#importing the store_id_relation and hpg_reserve data
store_id_relation = pd.read_csv("../input/store_id_relation.csv.zip")
hpg_reserve = pd.read_csv("../input/hpg_reserve.csv.zip")

convert_hpg = {k:v for k,v in list(zip(store_id_relation['hpg_store_id'].values,
                                       store_id_relation['air_store_id'].values))}
hpg_reserve["hpg_store_id"].replace(convert_hpg, inplace = True)
hpg_reserve = hpg_reserve[hpg_reserve['hpg_store_id'].str.startswith('air')]

Creating a new feature that indicate the delay between the reservation and the visit dates and merge the **air_reserve** and **hpg_reserve** dataframes in the **total_reserve** dataframe:

In [ ]:
def delta_reservation(df):
    df['delta_reservation'] = pd.to_datetime(df['visit_datetime']) - pd.to_datetime(df['reserve_datetime'])
    df['delta_2days'] = df['delta_reservation'].apply(lambda x: int(x.days < 2))
    df['delta_7days'] = df['delta_reservation'].apply(lambda x: int(2 <= x.days < 7))
    df['delta_long'] = df['delta_reservation'].apply(lambda x: int(x.days >= 7))
    return df

air_reserve = pd.read_csv("../input/air_reserve.csv.zip")
air_reserve = delta_reservation(air_reserve)
hpg_reserve = delta_reservation(hpg_reserve)

air_reserve.rename(columns = {'air_store_id':'store_id'}, inplace = True)
hpg_reserve.rename(columns = {'hpg_store_id':'store_id'}, inplace = True)
total_reserve = pd.concat([air_reserve, hpg_reserve])
total_reserve['date'] = total_reserve['visit_datetime'].apply(lambda x:x.date())

Considering the "air" restaurants, the **total_reserve** and **air_visit_data** dataframes gives info on the reservations and number of visits that were previously made. Taking a look at the number of unique restaurants in each dataframe:

In [ ]:
list_visit_ids   = air_visit_data['air_store_id'].unique()
list_reserve_ids = total_reserve['store_id'].unique()
print("nb. of restaurants visited: {}".format(len(list_visit_ids)))
print("nb. of restaurants with reservations: {}".format(len(list_reserve_ids)))
print("intersections of ids: {}".format(len(set(list_visit_ids).intersection(set(list_reserve_ids)))))

Around ~ 500 restaurants don't have any info on reservations.

### 3.1 A global view of all "air" restaurants

First, let's consider the overall visits and reservations made in the "air" restaurants.

In [ ]:
df1 = total_reserve[['date', 'reserve_visitors']].groupby('date').sum().reset_index()
df2 = air_visit_data.groupby('visit_date').sum().reset_index()

fig1 = Figure_style(11, 5, 1, 1)
fig1.pos_update(0, 0)
fig1.cust_plot(df2['visit_date'], df2['visitors'], linestyle='-', label = 'nb. of visits')
fig1.cust_plot(df1['date'], df1['reserve_visitors'], color = 'r', linestyle='-', label = 'nb. of reservations')
fig1.style() 
fig1.draw_legend(location = 'upper left')
fig1.set_ylabel('Visitors', fontsize = 14)
fig1.set_xlabel('Date', fontsize = 14)

#limits
date_1 = datetime.datetime(2015,12,1)
date_2 = datetime.datetime(2017,6,1)
fig1.set_xlim(date_1, date_2)
fig1.set_ylim(-50, 25000)

Here we can note a few things:
- the number of reservations only account for a small fraction of the total number of visits (typically a factor 10). This partly comes from the fact that only ~1/3 of the total number of restaurants visited are present in the **total_reserve** dataframe. Additionally, many clients will probably go directly to the restaurants without making a reservation beforehand.
- another interesting thing to note is that there is a clear high frequency peridocity and without looking at the data in details, we can infer that this periodicity arise from the day of the week, since it seems quite logical that the restaurant visits increases during the week-end.
- finally, **_we see that for the dates that correspond to the test set_**, the number of visits is quite low: this point will be adressed later but we can infer that for the 5 weeks covered by the test set, **_we don't have the data that deal with the 'last minute' reservations._**

___
### 3.2  Visualizing for a few select restaurants

Now, we shall examine the same thing but for a few selected restaurants.

#### 3.2.1 Case 1: the ideal case

We first select a restaurant in the **air_reserve** dataframe and compare the number of visits and number of reservations:

In [ ]:
restaurant_id = air_reserve['store_id'][0]

In [ ]:
df2 = air_visit_data[air_visit_data['air_store_id'] == restaurant_id]
df0 = total_reserve[total_reserve['store_id'] == restaurant_id]
df1 = df0[['date', 'reserve_visitors']].groupby('date').sum().reset_index()

In [ ]:
fig1 = Figure_style(11, 5, 1, 1)
fig1.pos_update(0, 0)
fig1.cust_plot(df2['visit_date'], df2['visitors'], linestyle='-', label = 'nb. of visits')
fig1.cust_plot(df1['date'], df1['reserve_visitors'], color = 'r', linestyle='-', label = 'nb. of reservations')
fig1.style() 
fig1.draw_legend(location = 'upper left')
fig1.set_ylabel('Visitors', fontsize = 14)
fig1.set_xlabel('Date', fontsize = 14)

#limits
date_1 = datetime.datetime(2015,12,21)
date_2 = datetime.datetime(2017,6,1)
fig1.set_xlim(date_1, date_2)
fig1.set_ylim(-3, 39)

Contrary to what we have seen so far, the number of reservations closely follows the numbers of visits in this case (when the reservations are available).

#### 3.2.2 Case 2

We select a second restaurant and proceed as before (except that I zoom on the data from Nov. 2016 to May 2017):

In [ ]:
restaurant_id = air_reserve['store_id'][2]

In [ ]:
df2 = air_visit_data[air_visit_data['air_store_id'] == restaurant_id]
df0 = total_reserve[total_reserve['store_id'] == restaurant_id]
df1 = df0[['date', 'reserve_visitors']].groupby('date').sum().reset_index()

In [ ]:
fig1 = Figure_style(11, 5, 1, 1)
fig1.pos_update(0, 0)
fig1.cust_plot(df2['visit_date'], df2['visitors'], linestyle='-', label = 'nb. of visits')
fig1.cust_plot(df1['date'], df1['reserve_visitors'], color = 'r', linestyle='-',
               marker = ['r', 'o', 5], label = 'nb. of reservations')
fig1.style() 
fig1.draw_legend(location = 'upper left')
fig1.set_ylabel('Visitors', fontsize = 14)
fig1.set_xlabel('Date', fontsize = 14)

#limits
date_1 = datetime.datetime(2016,11,1)
date_2 = datetime.datetime(2017,5,1)
fig1.set_xlim(date_1, date_2)
fig1.set_ylim(-3, 45)

Here, we see that number of visits *usually*  exceeds the number of reservations.

Hence, we will be faced with two categories of restaurants: for some restaurants, it will be easy to predict the number of visitors since they will only serve people that reserved beforehand. Other restaurants will on the contrary serve everybody.

### 3.3 Test set reservations

As commented earlier, there's a lack of information for the test set reservations. This is can be clearly seen in the graph below:

In [ ]:
fig1 = Figure_style(11, 5, 1, 1)
fig1.pos_update(0, 0)

color = ['r', 'b', 'g']
label = ['delay < 2 days', '2 days < delay < 7 days', 'delay > 7 days']
for j, colonne in enumerate(['delta_2days', 'delta_7days', 'delta_long']):
    df0 = total_reserve[total_reserve[colonne] == 1]
    df1 = df0[['date', 'reserve_visitors']].groupby('date').sum().reset_index()
    fig1.cust_plot(df1['date'], df1['reserve_visitors'], linestyle='-', label = label[j], color = color[j])

fig1.style() 
fig1.draw_legend(location = 'upper left')
fig1.set_ylabel('Visitors', fontsize = 14)
fig1.set_xlabel('Date', fontsize = 14)

#limits
date_1 = datetime.datetime(2017,2,1)
date_2 = datetime.datetime(2017,5,31)
fig1.set_xlim(date_1, date_2)
fig1.set_ylim(-3, 3000)
plt.show()

We can see that after April 22, the number of reservations starts to decline with time and after a few days, we don't have any information regarding the reservations made 2 days before the visit. In fact, after April 29, we don't have any info on the reservations made 7 days before the visit. Long term reservations are available for the test set, however their number tends to decrease with time.

The above observations might be crucial to making predictions in the test set since the public leaderboard would correspond to April 23rd - April 28th period, for which we still have some info concerning the number of reservations. If we take number of reservations into account, then the our model should perform reasonably well on the public leaderboard. However after April 28, there's a severe lack of info on the number of reservations and our models should take this into consideration as well, otherwise it will perform decently on the public leaderboard and poorly on the private leaderboard.